In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_r_raw = pd.read_csv(url_cum)
df_r_raw

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5434,2020-11-08T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1250,177,1427,20040,...,730,1083,9128,676,19378.0,11893.0,31271,753094,525013.0,NaN
5435,2020-11-08T17:00:00,ITA,9,Toscana,43.769231,11.255889,1474,226,1700,41747,...,1864,2479,17938,1561,50905.0,12041.0,62946,1214035,799345.0,NaN
5436,2020-11-08T17:00:00,ITA,10,Umbria,43.106758,12.388247,353,62,415,9385,...,430,660,4951,188,4189.0,10750.0,14939,329812,189320.0,NaN
5437,2020-11-08T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,158,14,172,2000,...,-9,55,1758,205,3634.0,501.0,4135,45323,27399.0,NaN


In [4]:
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_r_raw.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5434,2020-11-08T17:00:00,19,Sicilia,1250,177,1427,20040,21467,730,1083,9128,676,19378.0,11893.0,31271,753094,525013.0
5435,2020-11-08T17:00:00,9,Toscana,1474,226,1700,41747,43447,1864,2479,17938,1561,50905.0,12041.0,62946,1214035,799345.0
5436,2020-11-08T17:00:00,10,Umbria,353,62,415,9385,9800,430,660,4951,188,4189.0,10750.0,14939,329812,189320.0
5437,2020-11-08T17:00:00,2,Valle d'Aosta,158,14,172,2000,2172,-9,55,1758,205,3634.0,501.0,4135,45323,27399.0


In [5]:
# Current state S
df_raw_r_1 = df_rd[-21:].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-08T17:00:00,1,Piemonte,4367,304,4671,49526,54197,2708,3884,38953,4629,42321.0,55458.0,97779,1149489,726199.0
1,2020-11-08T17:00:00,2,Valle d'Aosta,158,14,172,2000,2172,-9,55,1758,205,3634.0,501.0,4135,45323,27399.0
2,2020-11-08T17:00:00,3,Lombardia,6225,650,6875,125535,132410,4781,6318,110001,18343,201314.0,59440.0,260754,3256646,2003397.0
3,2020-11-08T17:00:00,5,Veneto,1355,185,1540,49430,50970,3313,3362,25725,2574,26082.0,53187.0,79269,2448503,948701.0
4,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301,45,346,7295,7641,281,504,6636,451,12787.0,1941.0,14728,580815,242297.0
5,2020-11-08T17:00:00,7,Liguria,1334,81,1415,10993,12408,300,886,22106,1916,25326.0,11104.0,36430,486171,248706.0
6,2020-11-08T17:00:00,8,Emilia-Romagna,1836,194,2030,35822,37852,2203,2360,28740,4816,48938.0,22470.0,71408,1728477,914731.0
7,2020-11-08T17:00:00,9,Toscana,1474,226,1700,41747,43447,1864,2479,17938,1561,50905.0,12041.0,62946,1214035,799345.0
8,2020-11-08T17:00:00,10,Umbria,353,62,415,9385,9800,430,660,4951,188,4189.0,10750.0,14939,329812,189320.0
9,2020-11-08T17:00:00,11,Marche,493,69,562,9679,10241,414,502,7498,1050,18600.0,189.0,18789,345496,202915.0


In [6]:
# State S - 1
df_raw_r_0 = df_rd[-42:-21].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-07T17:00:00,1,Piemonte,4122,284,4406,47083,51489,3573,4437,37812,4594,40806.0,53089.0,93895,1134670,715993.0
1,2020-11-07T17:00:00,2,Valle d'Aosta,154,14,168,2013,2181,13,117,1697,202,3584.0,496.0,4080,44845,27267.0
2,2020-11-07T17:00:00,3,Lombardia,5813,610,6423,121206,127629,10263,11489,108581,18226,197252.0,57184.0,254436,3218458,1981508.0
3,2020-11-07T17:00:00,5,Veneto,1300,176,1476,46181,47657,3720,3815,25682,2568,25971.0,49936.0,75907,2437080,944143.0
4,2020-11-07T17:00:00,6,Friuli Venezia Giulia,286,45,331,7029,7360,605,872,6422,442,12386.0,1838.0,14224,576329,240788.0
5,2020-11-07T17:00:00,7,Liguria,1303,77,1380,10728,12108,416,1092,21522,1914,24746.0,10798.0,35544,481315,246833.0
6,2020-11-07T17:00:00,8,Emilia-Romagna,1723,188,1911,33738,35649,1919,2009,28619,4781,48031.0,21018.0,69049,1715527,906435.0
7,2020-11-07T17:00:00,9,Toscana,1368,214,1582,40001,41583,2143,2787,17352,1532,48875.0,11592.0,60467,1198493,791081.0
8,2020-11-07T17:00:00,10,Umbria,345,58,403,8967,9370,485,688,4731,178,4029.0,10250.0,14279,325727,187400.0
9,2020-11-07T17:00:00,11,Marche,454,67,521,9306,9827,587,631,7415,1045,18037.0,250.0,18287,342550,201310.0


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-08T17:00:00,1,Piemonte,4367.0,304.0,4671.0,49526.0,54197.0,2708.0,3884.0,38953.0,4629.0,42321.0,55458.0,97779.0,1149489.0,726199.0
1,2020-11-08T17:00:00,2,Valle d'Aosta,158.0,14.0,172.0,2000.0,2172.0,-9.0,55.0,1758.0,205.0,3634.0,501.0,4135.0,45323.0,27399.0
2,2020-11-08T17:00:00,3,Lombardia,6225.0,650.0,6875.0,125535.0,132410.0,4781.0,6318.0,110001.0,18343.0,201314.0,59440.0,260754.0,3256646.0,2003397.0
3,2020-11-08T17:00:00,4,Trentino-Alto Adige,653.0,59.0,712.0,10028.0,10740.0,642.0,963.0,12466.0,806.0,19130.0,4882.0,24012.0,565246.0,NaN
4,2020-11-08T17:00:00,5,Veneto,1355.0,185.0,1540.0,49430.0,50970.0,3313.0,3362.0,25725.0,2574.0,26082.0,53187.0,79269.0,2448503.0,948701.0
5,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301.0,45.0,346.0,7295.0,7641.0,281.0,504.0,6636.0,451.0,12787.0,1941.0,14728.0,580815.0,242297.0
6,2020-11-08T17:00:00,7,Liguria,1334.0,81.0,1415.0,10993.0,12408.0,300.0,886.0,22106.0,1916.0,25326.0,11104.0,36430.0,486171.0,248706.0
7,2020-11-08T17:00:00,8,Emilia-Romagna,1836.0,194.0,2030.0,35822.0,37852.0,2203.0,2360.0,28740.0,4816.0,48938.0,22470.0,71408.0,1728477.0,914731.0
8,2020-11-08T17:00:00,9,Toscana,1474.0,226.0,1700.0,41747.0,43447.0,1864.0,2479.0,17938.0,1561.0,50905.0,12041.0,62946.0,1214035.0,799345.0
9,2020-11-08T17:00:00,10,Umbria,353.0,62.0,415.0,9385.0,9800.0,430.0,660.0,4951.0,188.0,4189.0,10750.0,14939.0,329812.0,189320.0


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-07T17:00:00,1,Piemonte,4122.0,284.0,4406.0,47083.0,51489.0,3573.0,4437.0,37812.0,4594.0,40806.0,53089.0,93895.0,1134670.0,715993.0
1,1,2020-11-07T17:00:00,2,Valle d'Aosta,154.0,14.0,168.0,2013.0,2181.0,13.0,117.0,1697.0,202.0,3584.0,496.0,4080.0,44845.0,27267.0
2,2,2020-11-07T17:00:00,3,Lombardia,5813.0,610.0,6423.0,121206.0,127629.0,10263.0,11489.0,108581.0,18226.0,197252.0,57184.0,254436.0,3218458.0,1981508.0
3,19,2020-11-07T17:00:00,4,Trentino-Alto Adige,604.0,54.0,658.0,9440.0,10098.0,619.0,1024.0,12150.0,801.0,18280.0,4769.0,23049.0,560589.0,NaN
4,3,2020-11-07T17:00:00,5,Veneto,1300.0,176.0,1476.0,46181.0,47657.0,3720.0,3815.0,25682.0,2568.0,25971.0,49936.0,75907.0,2437080.0,944143.0
5,4,2020-11-07T17:00:00,6,Friuli Venezia Giulia,286.0,45.0,331.0,7029.0,7360.0,605.0,872.0,6422.0,442.0,12386.0,1838.0,14224.0,576329.0,240788.0
6,5,2020-11-07T17:00:00,7,Liguria,1303.0,77.0,1380.0,10728.0,12108.0,416.0,1092.0,21522.0,1914.0,24746.0,10798.0,35544.0,481315.0,246833.0
7,6,2020-11-07T17:00:00,8,Emilia-Romagna,1723.0,188.0,1911.0,33738.0,35649.0,1919.0,2009.0,28619.0,4781.0,48031.0,21018.0,69049.0,1715527.0,906435.0
8,7,2020-11-07T17:00:00,9,Toscana,1368.0,214.0,1582.0,40001.0,41583.0,2143.0,2787.0,17352.0,1532.0,48875.0,11592.0,60467.0,1198493.0,791081.0
9,8,2020-11-07T17:00:00,10,Umbria,345.0,58.0,403.0,8967.0,9370.0,485.0,688.0,4731.0,178.0,4029.0,10250.0,14279.0,325727.0,187400.0


In [9]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-08T17:00:00,1,Piemonte,4367.0,304.0,4671.0,49526.0,54197.0,2708.0,3884.0,38953.0,4629.0,42321.0,55458.0,97779.0,1149489.0,726199.0,4341375
1,2020-11-08T17:00:00,2,Valle d'Aosta,158.0,14.0,172.0,2000.0,2172.0,-9.0,55.0,1758.0,205.0,3634.0,501.0,4135.0,45323.0,27399.0,125501
2,2020-11-08T17:00:00,3,Lombardia,6225.0,650.0,6875.0,125535.0,132410.0,4781.0,6318.0,110001.0,18343.0,201314.0,59440.0,260754.0,3256646.0,2003397.0,10103969
3,2020-11-08T17:00:00,4,Trentino-Alto Adige,653.0,59.0,712.0,10028.0,10740.0,642.0,963.0,12466.0,806.0,19130.0,4882.0,24012.0,565246.0,NaN,1074819
4,2020-11-08T17:00:00,5,Veneto,1355.0,185.0,1540.0,49430.0,50970.0,3313.0,3362.0,25725.0,2574.0,26082.0,53187.0,79269.0,2448503.0,948701.0,4907704
5,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301.0,45.0,346.0,7295.0,7641.0,281.0,504.0,6636.0,451.0,12787.0,1941.0,14728.0,580815.0,242297.0,1211357
6,2020-11-08T17:00:00,7,Liguria,1334.0,81.0,1415.0,10993.0,12408.0,300.0,886.0,22106.0,1916.0,25326.0,11104.0,36430.0,486171.0,248706.0,1543127
7,2020-11-08T17:00:00,8,Emilia-Romagna,1836.0,194.0,2030.0,35822.0,37852.0,2203.0,2360.0,28740.0,4816.0,48938.0,22470.0,71408.0,1728477.0,914731.0,4467118
8,2020-11-08T17:00:00,9,Toscana,1474.0,226.0,1700.0,41747.0,43447.0,1864.0,2479.0,17938.0,1561.0,50905.0,12041.0,62946.0,1214035.0,799345.0,3722729
9,2020-11-08T17:00:00,10,Umbria,353.0,62.0,415.0,9385.0,9800.0,430.0,660.0,4951.0,188.0,4189.0,10750.0,14939.0,329812.0,189320.0,880285


In [10]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-08T17:00:00,1,Piemonte,4367.0,304.0,4671.0,49526.0,54197.0,2708.0,3884.0,38953.0,4629.0,42321.0,55458.0,97779.0,1149489.0,726199.0,4341375,367
1,2020-11-08T17:00:00,2,Valle d'Aosta,158.0,14.0,172.0,2000.0,2172.0,-9.0,55.0,1758.0,205.0,3634.0,501.0,4135.0,45323.0,27399.0,125501,20
2,2020-11-08T17:00:00,3,Lombardia,6225.0,650.0,6875.0,125535.0,132410.0,4781.0,6318.0,110001.0,18343.0,201314.0,59440.0,260754.0,3256646.0,2003397.0,10103969,983
3,2020-11-08T17:00:00,4,Trentino-Alto Adige,653.0,59.0,712.0,10028.0,10740.0,642.0,963.0,12466.0,806.0,19130.0,4882.0,24012.0,565246.0,NaN,1074819,106
4,2020-11-08T17:00:00,5,Veneto,1355.0,185.0,1540.0,49430.0,50970.0,3313.0,3362.0,25725.0,2574.0,26082.0,53187.0,79269.0,2448503.0,948701.0,4907704,825
5,2020-11-08T17:00:00,6,Friuli Venezia Giulia,301.0,45.0,346.0,7295.0,7641.0,281.0,504.0,6636.0,451.0,12787.0,1941.0,14728.0,580815.0,242297.0,1211357,175
6,2020-11-08T17:00:00,7,Liguria,1334.0,81.0,1415.0,10993.0,12408.0,300.0,886.0,22106.0,1916.0,25326.0,11104.0,36430.0,486171.0,248706.0,1543127,209
7,2020-11-08T17:00:00,8,Emilia-Romagna,1836.0,194.0,2030.0,35822.0,37852.0,2203.0,2360.0,28740.0,4816.0,48938.0,22470.0,71408.0,1728477.0,914731.0,4467118,516
8,2020-11-08T17:00:00,9,Toscana,1474.0,226.0,1700.0,41747.0,43447.0,1864.0,2479.0,17938.0,1561.0,50905.0,12041.0,62946.0,1214035.0,799345.0,3722729,415
9,2020-11-08T17:00:00,10,Umbria,353.0,62.0,415.0,9385.0,9800.0,430.0,660.0,4951.0,188.0,4189.0,10750.0,14939.0,329812.0,189320.0,880285,70


In [11]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [12]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,saturazione_terapia_intensiva,totale_posti_terapia_intensiva
0,Lombardia,650.0,333.0,66.12,983
1,Campania,186.0,241.0,43.56,427
2,Piemonte,304.0,63.0,82.83,367
3,Veneto,185.0,640.0,22.42,825
4,Lazio,237.0,510.0,31.73,747
5,Toscana,226.0,189.0,54.46,415
6,Emilia-Romagna,194.0,322.0,37.60,516
7,Sicilia,177.0,361.0,32.90,538
8,Trentino-Alto Adige,59.0,47.0,55.66,106
9,Liguria,81.0,128.0,38.76,209


In [13]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [16]:
# ICU Stats
df_r[['denominazione_regione', 'totale_posti_terapia_intensiva', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,totale_posti_terapia_intensiva,terapia_intensiva,posti_disponibili_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,70,62.0,8.0,88.57
1,Piemonte,367,304.0,63.0,82.83
2,Valle d'Aosta,20,14.0,6.0,70.00
3,Lombardia,983,650.0,333.0,66.12
4,Trentino-Alto Adige,106,59.0,47.0,55.66
5,Toscana,415,226.0,189.0,54.46
6,Marche,127,69.0,58.0,54.33
7,Campania,427,186.0,241.0,43.56
8,Liguria,209,81.0,128.0,38.76
9,Emilia-Romagna,516,194.0,322.0,37.60
